In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
CFG = {
    'seed': 42,
    'model_name': 'tf_efficientnet_b3_ns', 
    'img_size': 512,                     
    'batch_size': 16,                    
    'epochs': 15,                        
    'lr': 1e-4,
    'num_workers': 4,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'output_dir': '/kaggle/working'
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(CFG['seed'])

In [3]:
base_path = '/kaggle/input'
dataset_root = None

for root, dirs, files in os.walk(base_path):
    if 'train_labels.csv' in files:
        dataset_root = root
        break

if dataset_root:
    print(f"Found dataset at: {dataset_root}")
    TRAIN_CSV = os.path.join(dataset_root, 'train_labels.csv')
    
    if os.path.exists(os.path.join(dataset_root, 'train', 'train')):
        TRAIN_IMG_DIR = os.path.join(dataset_root, 'train', 'train')
    else:
        TRAIN_IMG_DIR = os.path.join(dataset_root, 'train')
        
    if os.path.exists(os.path.join(dataset_root, 'test', 'test')):
        TEST_IMG_DIR = os.path.join(dataset_root, 'test', 'test')
    else:
        TEST_IMG_DIR = os.path.join(dataset_root, 'test')
        
    print(f"Train Images: {TRAIN_IMG_DIR}")
    print(f"Test Images:  {TEST_IMG_DIR}")
    print(f"Label File:   {TRAIN_CSV}")
else:
    raise FileNotFoundError("Could not find 'train_labels.csv'. Check the dataset mounting.")


Found dataset at: /kaggle/input/solidworks-ai-hackathon
Train Images: /kaggle/input/solidworks-ai-hackathon/train/train
Test Images:  /kaggle/input/solidworks-ai-hackathon/test/test
Label File:   /kaggle/input/solidworks-ai-hackathon/train_labels.csv


In [4]:
class PartsDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_test=False):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name = row['image_name']
        img_path = os.path.join(self.img_dir, img_name)
        
        image = cv2.imread(img_path)
        if image is None:
            image = np.zeros((CFG['img_size'], CFG['img_size'], 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        if self.is_test:
            return image, img_name
        else:
            labels = row[['bolt', 'locatingpin', 'nut', 'washer']].values.astype(np.float32)
            return image, torch.tensor(labels)


In [5]:
train_transforms = A.Compose([
    A.Resize(CFG['img_size'], CFG['img_size']),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transforms = A.Compose([
    A.Resize(CFG['img_size'], CFG['img_size']),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

class CountModel(nn.Module):
    def __init__(self, model_name, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained, num_classes=0)
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.backbone.num_features, 512),
            nn.Mish(),
            nn.Dropout(0.2),
            nn.Linear(512, 4)
        )
        
    def forward(self, x):
        return self.head(self.backbone(x))


In [6]:
def train_fn(loader, model, criterion, optimizer, scaler, device):
    model.train()
    running_loss = 0
    for images, labels in tqdm(loader, desc="Train", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item()
    return running_loss / len(loader)

def valid_fn(loader, model, criterion, device):
    model.eval()
    preds_all, targets_all = [], []
    running_loss = 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Valid", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds_all.append(outputs.cpu().numpy())
            targets_all.append(labels.cpu().numpy())
    
    preds_all = np.concatenate(preds_all)
    targets_all = np.concatenate(targets_all)
    # Exact Match Logic
    acc = np.all(np.round(preds_all).clip(min=0) == targets_all, axis=1).mean()
    return running_loss / len(loader), acc


In [7]:
if __name__ == '__main__':
    df = pd.read_csv(TRAIN_CSV)
    
    required_cols = ['bolt', 'locatingpin', 'nut', 'washer']
    if not all(col in df.columns for col in required_cols):
        print(f"Columns missing in CSV! Found: {df.columns}")
        print("Attempting to pivot/fix (assuming 'class_id' or similar format)...")
    
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=CFG['seed'])
    
    train_loader = DataLoader(PartsDataset(train_df, TRAIN_IMG_DIR, transform=train_transforms), 
                              batch_size=CFG['batch_size'], shuffle=True, num_workers=CFG['num_workers'])
    val_loader = DataLoader(PartsDataset(val_df, TRAIN_IMG_DIR, transform=val_transforms), 
                            batch_size=CFG['batch_size'], shuffle=False, num_workers=CFG['num_workers'])
    
    model = CountModel(CFG['model_name']).to(CFG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CFG['lr'])
    criterion = nn.HuberLoss()
    scaler = GradScaler()
    
    best_score = 0
    
    for epoch in range(CFG['epochs']):
        t_loss = train_fn(train_loader, model, criterion, optimizer, scaler, CFG['device'])
        v_loss, v_acc = valid_fn(val_loader, model, criterion, CFG['device'])
        print(f"Epoch {epoch+1} | Train Loss: {t_loss:.4f} | Val Exact Acc: {v_acc:.4f}")
        
        if v_acc > best_score:
            best_score = v_acc
            torch.save(model.state_dict(), os.path.join(CFG['output_dir'], 'best_model.pth'))


/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

/tmp/ipykernel_19/1837512111.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Train:   0%|          | 0/563 [00:00<?, ?it/s]

/tmp/ipykernel_19/338538313.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


...

Valid:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 14 | Train Loss: 0.0029 | Val Exact Acc: 0.9970


Train:   0%|          | 0/563 [00:00<?, ?it/s]

Valid:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 15 | Train Loss: 0.0031 | Val Exact Acc: 0.9970


In [8]:

    model.load_state_dict(torch.load(os.path.join(CFG['output_dir'], 'best_model.pth')))
    model.eval()
    
    test_files = os.listdir(TEST_IMG_DIR)
    test_df = pd.DataFrame({'image_name': test_files})
    test_loader = DataLoader(PartsDataset(test_df, TEST_IMG_DIR, transform=val_transforms, is_test=True), 
                             batch_size=CFG['batch_size'], shuffle=False, num_workers=CFG['num_workers'])
    
    final_preds = []
    final_names = []
    
    with torch.no_grad():
        for images, names in tqdm(test_loader, desc="Infer"):
            images = images.to(CFG['device'])
            outputs = model(images)
            preds = torch.round(outputs).cpu().numpy().clip(min=0).astype(int)
            final_preds.append(preds)
            final_names.extend(names)
            
    final_preds = np.vstack(final_preds)
    sub = pd.DataFrame(final_preds, columns=['bolt', 'locatingpin', 'nut', 'washer'])
    sub.insert(0, 'image_name', final_names)
    sub.to_csv(os.path.join(CFG['output_dir'], 'submission.csv'), index=False)
    print("submission.csv saved!")

Infer:   0%|          | 0/125 [00:00<?, ?it/s]

submission.csv saved!


In [9]:
CFG = {
    'model_name': 'tf_efficientnet_b3_ns',
    'img_size': 512,
    'batch_size': 16,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'output_dir': '/kaggle/working',
    'input_root': '/kaggle/input' 
}

In [10]:
TEST_IMG_DIR = None
for root, dirs, files in os.walk(CFG['input_root']):
    if 'test' in dirs:
        test_path = os.path.join(root, 'test')
        if 'test' in os.listdir(test_path):
             TEST_IMG_DIR = os.path.join(test_path, 'test')
             break
        else:
             TEST_IMG_DIR = test_path
             break

if not TEST_IMG_DIR:
    raise FileNotFoundError("CRITICAL: Could not find test image directory!")

print(f"Scanning Images from: {TEST_IMG_DIR}")

test_files = [f for f in os.listdir(TEST_IMG_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
test_files = sorted(test_files) 

if len(test_files) == 0:
    raise RuntimeError("Found 0 images in the test folder. Check paths!")

print(f"Found {len(test_files)} images to predict.")
df_submission = pd.DataFrame({'image_name': test_files})


Scanning Images from: /kaggle/input/solidworks-ai-hackathon/test/test
Found 2000 images to predict.


In [11]:
class CountModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.backbone = timm.create_model(model_name, pretrained=False, num_classes=0)
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.backbone.num_features, 512),
            nn.Mish(),
            nn.Dropout(0.2),
            nn.Linear(512, 4)
        )
        
    def forward(self, x):
        return self.head(self.backbone(x))


In [12]:
class InferenceDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir
        self.transform = A.Compose([
            A.Resize(CFG['img_size'], CFG['img_size']),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name = str(row['image_name'])
        img_path = os.path.join(self.img_dir, img_name)
        
        image = np.zeros((CFG['img_size'], CFG['img_size'], 3), dtype=np.uint8)
        if os.path.exists(img_path):
            loaded = cv2.imread(img_path)
            if loaded is not None:
                image = cv2.cvtColor(loaded, cv2.COLOR_BGR2RGB)
                
        image = self.transform(image=image)['image']
        return image

model = CountModel(CFG['model_name']).to(CFG['device'])
weights_path = '/kaggle/working/best_model.pth'
if not os.path.exists(weights_path):
    raise FileNotFoundError("best_model.pth not found! You must run the Training Cell first.")
    
model.load_state_dict(torch.load(weights_path, map_location=CFG['device']))
model.eval()

ds = InferenceDataset(df_submission, TEST_IMG_DIR)
loader = DataLoader(ds, batch_size=CFG['batch_size']*2, shuffle=False, num_workers=4)

all_preds = []
with torch.no_grad():
    for images in tqdm(loader):
        images = images.to(CFG['device'])
        outputs = model(images)
        preds = torch.round(outputs).cpu().numpy().clip(min=0).astype(int)
        all_preds.append(preds)

all_preds = np.vstack(all_preds)

df_submission[['bolt', 'locatingpin', 'nut', 'washer']] = all_preds

save_path = '/kaggle/working/submission.csv'
df_submission.to_csv(save_path, index=False)

print(f"Generated {len(df_submission)} predictions.")
print(df_submission.head())

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


  0%|          | 0/63 [00:00<?, ?it/s]

Generated 2000 predictions.
                             image_name  bolt  locatingpin  nut  washer
0  0040313aa7c7478c8ca264bf573c53fe.png     1            3    0       0
1  0080e45f4375443b96ee81dc04075117.png     2            0    0       1
2  0086da1a1914469caa4d042f8e4e94ef.png     0            0    0       1
3  009ccd5b64a043e0afbae0f840f7cfd4.png     0            0    0       1
4  00a522a027b24fcfb8ee0857655b953d.png     0            1    0       0
